## Imports & Model Loading

In [ ]:
# ======================
# Core
# ======================
import os
import math
import json
import random
from pathlib import Path

# ======================
# Numerical / CV
# ======================
import numpy as np
import cv2
import matplotlib.pyplot as plt

# ======================
# Torch (for later models)
# ======================
import torch
import torchvision

# ======================
# Colab utilities
# ======================
from google.colab import files
from IPython.display import display

# ======================
# Plot settings
# ======================
plt.rcParams["figure.figsize"] = (6, 6)
plt.rcParams["image.cmap"] = "viridis"

print("✅ Imports loaded")
print(f"🔥 Torch CUDA available: {torch.cuda.is_available()}")

In [ ]:
import torch
import cv2
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

model_type = "DPT_Large"  # best quality; use DPT_Hybrid if memory issues

MIDAS = torch.hub.load("intel-isl/MiDaS", model_type)
MIDAS.to(device)
MIDAS.eval()

transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
MIDAS_TRANSFORM = transforms.dpt_transform

## Steps: Helpers

In [ ]:
# Random Helpers
def show_image(img, title=""):
    plt.figure()
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()

import numpy as np
import plotly.graph_objects as go

def plot_heatmap_overlay(data, img1=None, mask=None):
    """
    Plot a semi-transparent heatmap over an optional background image.

    Args:
        data: (H, W) scalar array
        img1: optional (H, W, 3) background image
        mask: optional (H, W) mask (truthy = keep)
    """
    data = np.asarray(data)

    if data.ndim != 2:
        raise ValueError(f"`data` must be 2D. Got {data.shape}")

    H, W = data.shape

    # Validate image
    if img1 is not None:
        img1 = np.asarray(img1)
        if img1.shape[:2] != (H, W):
            raise ValueError("`img1` must match data spatial dimensions")

    # Apply mask
    if mask is not None:
        mask = np.asarray(mask)
        if mask.shape != (H, W):
            raise ValueError("`mask` must match data shape")
        data = np.where(mask.astype(bool), data, np.nan)

    fig = go.Figure()

    # Background
    if img1 is not None:
        fig.add_trace(go.Image(z=img1))

    # Heatmap
    fig.add_trace(
        go.Heatmap(
            z=data,
            colorscale="Viridis",
            opacity=0.45,
            hovertemplate="x=%{x}<br>y=%{y}<br>value=%{z:.2f}<extra></extra>",
            colorbar=dict(title="value"),
            zsmooth=False,
        )
    )

    fig.update_layout(
        title="Heatmap Overlay",
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False, autorange="reversed"),
        width=700,
        height=700,
    )

    return fig

### Step 1: Image preprocessing

In [ ]:
def read_image(path, rgb=True):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"❌ Failed to read image: {path}")
    if rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


def resize_max_dim(img, max_dim):
    h, w = img.shape[:2]
    scale = max_dim / max(h, w)

    if scale >= 1:
        return img, 1.0

    new_w = int(w * scale)
    new_h = int(h * scale)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized, scale

def s1_upload_preprocess_image():
  print("Upload ONE plate image")
  uploaded = files.upload()

  # Basic validation
  if len(uploaded) < 1:
      raise ValueError("❌ Please upload one image.")

  # Sort for deterministic order
  image_paths = sorted(uploaded.keys())

  IMG1_PATH = image_paths[0]

  img1 = read_image(IMG1_PATH)

  img1, scale1 = resize_max_dim(img1, max_dim=1024)

  return img1


### Step 2: Detect plate

In [ ]:
import numpy as np
import cv2
import math

def overlay_mask(img, mask, alpha=0.35):
    color = np.zeros_like(img)
    color[..., 1] = 255  # green overlay
    return np.where(mask[..., None] > 0,
                    (img * (1-alpha) + color * alpha).astype(np.uint8),
                    img)

def s2_detect_plate_mask(image_rgb: np.ndarray) -> np.ndarray:
    """
    Detect the main plate as a filled ellipse mask.

    Input:
        image_rgb: HxWx3 RGB image

    Output:
        mask: HxW uint8 mask (255 = plate, 0 = background)

    Assumptions:
        - plate is fully visible
        - plate is roughly circular
        - rim has edge contrast
    """

    h, w = image_rgb.shape[:2]

    # ---------- 1. edge map ----------
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    edges = cv2.Canny(gray, 60, 160)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # ---------- 2. contours ----------
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if not contours:
        return np.zeros((h, w), dtype=np.uint8)

    img_area = h * w
    best_score = -1
    best_ellipse = None

    # ---------- 3. evaluate candidates ----------
    for cnt in contours:
        if len(cnt) < 80:
            continue

        area = cv2.contourArea(cnt)
        if area < 0.01 * img_area:
            continue

        try:
            ellipse = cv2.fitEllipse(cnt)
        except:
            continue

        (cx, cy), (a, b), angle = ellipse
        major = max(a, b)
        minor = min(a, b)

        # reject extremely skinny ellipses
        if minor / major < 0.4:
            continue

        # ---------- rim support scoring ----------
        # sample points along ellipse perimeter
        num_samples = 360
        theta = np.linspace(0, 2*np.pi, num_samples)

        cos_a = math.cos(math.radians(angle))
        sin_a = math.sin(math.radians(angle))

        rx = major / 2
        ry = minor / 2

        hits = 0
        valid = 0

        for t in theta:
            x = rx * np.cos(t)
            y = ry * np.sin(t)

            # rotate
            xr = x * cos_a - y * sin_a
            yr = x * sin_a + y * cos_a

            px = int(cx + xr)
            py = int(cy + yr)

            if 0 <= px < w and 0 <= py < h:
                valid += 1

                # small neighborhood check for robustness
                x0 = max(px - 2, 0)
                x1 = min(px + 3, w)
                y0 = max(py - 2, 0)
                y1 = min(py + 3, h)

                if np.any(edges[y0:y1, x0:x1] > 0):
                    hits += 1

        if valid == 0:
            continue

        support_ratio = hits / valid

        # slight size prior (plate should be reasonably large)
        size_score = min(1.0, major / max(h, w))

        score = support_ratio * (0.7 + 0.3 * size_score)

        if score > best_score:
            best_score = score
            best_ellipse = ellipse

    # ---------- 4. build mask ----------
    mask = np.zeros((h, w), dtype=np.uint8)

    if best_ellipse is not None:
        cv2.ellipse(mask, best_ellipse, 255, thickness=-1)

    return mask

#fig = plot_heatmap_overlay(Z_abs_mm, img1=img1, mask=mask)
#fig.show()

### Step 3: Create empty plate relative height model

In [ ]:
def _to_u8_mask(m):
    m = (m > 0).astype(np.uint8)
    return m

def ellipse_params_from_mask(plate_mask_u8):
    """
    Returns (cx, cy, plate_diameter_px, (ellipse_center, axes, angle_deg))
    where plate_diameter_px uses the major axis length (in px).
    """
    cnts, _ = cv2.findContours(plate_mask_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        raise RuntimeError("No contour found in plate_mask.")
    c = max(cnts, key=cv2.contourArea)
    if len(c) < 5:
        raise RuntimeError("Not enough contour points to fit ellipse.")
    (cx, cy), (w, h), angle = cv2.fitEllipse(c)
    plate_diameter_px = float(max(w, h))  # major axis
    return (float(cx), float(cy), plate_diameter_px, ((cx, cy), (w, h), angle))

import numpy as np

def s3_empty_plate_height_map(
    *,
    height_px: int,
    width_px: int,
    center_xy_px: tuple[float, float],  # (cx, cy)

    # plate size in the image (px)
    plate_diameter_px: float,

    # Plate geometry (mm)
    outer_diameter_mm: float,
    inner_radius_mm: float,
    lip_end_radius_mm: float,
    slope_end_radius_mm: float,

    # Heights above table (mm)
    h_inner_mm: float = 0.0,
    h_lip_mm: float | None = None,
    h_slope_end_mm: float = 0.0,

    # Profiles
    lip_profile: str = "smooth",
    slope_profile: str = "smooth",

    feather_px: float = 2.0,
    eps: float = 1e-6,
):
    """
    Returns:
        Z_rel : (H,W) float32 height map normalized to [0,1],
                where table/outside plate = 0 and highest plate point = 1.
    """

    if h_lip_mm is None:
        h_lip_mm = h_inner_mm

    # --------------------------------------------------
    # Derived metric scale
    # --------------------------------------------------
    mm_per_px = outer_diameter_mm / plate_diameter_px
    outer_radius_mm = outer_diameter_mm / 2.0

    if not (0 <= inner_radius_mm <= lip_end_radius_mm <= slope_end_radius_mm <= outer_radius_mm):
        raise ValueError("Radii must satisfy: 0 <= inner <= lip_end <= slope_end <= outer.")

    cx, cy = center_xy_px

    # --------------------------------------------------
    # Pixel grid
    # --------------------------------------------------
    ys, xs = np.indices((height_px, width_px), dtype=np.float32)
    dx_px = xs - cx
    dy_px = ys - cy
    r_mm = np.sqrt((dx_px * mm_per_px) ** 2 + (dy_px * mm_per_px) ** 2)

    # --------------------------------------------------
    # Helpers
    # --------------------------------------------------
    def smoothstep01(t):
        t = np.clip(t, 0.0, 1.0)
        return t * t * (3.0 - 2.0 * t)

    def ramp(t, mode):
        if mode == "smooth":
            return smoothstep01(t)
        if mode == "linear":
            return np.clip(t, 0.0, 1.0)
        if mode == "flat":
            return (t >= 0).astype(np.float32)
        raise ValueError("profile must be 'smooth', 'linear', or 'flat'")

    def soft_inside(r, R):
        if feather_px <= 0:
            return (r <= R).astype(np.float32)
        feather_mm = feather_px * mm_per_px
        lo = R - feather_mm / 2.0
        hi = R + feather_mm / 2.0
        t = (hi - r) / (hi - lo)
        return np.clip(t, 0.0, 1.0).astype(np.float32)

    # --------------------------------------------------
    # Soft region gates
    # --------------------------------------------------
    inside_inner = soft_inside(r_mm, inner_radius_mm)
    inside_lip_end = soft_inside(r_mm, lip_end_radius_mm)
    inside_slope_end = soft_inside(r_mm, slope_end_radius_mm)
    inside_outer = soft_inside(r_mm, outer_radius_mm)

    lip_ring = np.clip(inside_lip_end - inside_inner, 0.0, 1.0)
    slope_ring = np.clip(inside_slope_end - inside_lip_end, 0.0, 1.0)
    outer_band = np.clip(inside_outer - inside_slope_end, 0.0, 1.0)

    # --------------------------------------------------
    # Height construction (mm)
    # --------------------------------------------------
    Z_mm = np.zeros((height_px, width_px), dtype=np.float32)

    # inner flat
    Z_mm += h_inner_mm * inside_inner

    # lip transition
    if lip_end_radius_mm > inner_radius_mm:
        t = (r_mm - inner_radius_mm) / (lip_end_radius_mm - inner_radius_mm)
        s = ramp(t, lip_profile)
        Z_lip = h_inner_mm + (h_lip_mm - h_inner_mm) * s
        Z_mm += Z_lip * lip_ring

    # slope transition
    if slope_end_radius_mm > lip_end_radius_mm:
        t = (r_mm - lip_end_radius_mm) / (slope_end_radius_mm - lip_end_radius_mm)
        s = ramp(t, slope_profile)
        Z_slope = h_lip_mm + (h_slope_end_mm - h_lip_mm) * s
        Z_mm += Z_slope * slope_ring

    # outer rim band
    Z_mm += h_slope_end_mm * outer_band

    # zero outside plate (table)
    Z_mm *= inside_outer

    # --------------------------------------------------
    # Normalize to relative [0,1]
    # --------------------------------------------------
    # Table already at 0. Find the max height on the plate (use inside_outer as mask).
    zmax = float(Z_mm.max())
    if zmax <= eps:
        return np.zeros_like(Z_mm, dtype=np.float32)

    Z_rel = (Z_mm / zmax).astype(np.float32)

    # keep outside at exactly 0 (avoid tiny numeric bleed in feather region)
    Z_rel *= inside_outer

    return Z_mm, Z_rel

## Step 4: Run MiDas

In [ ]:
def s4_run_midas(midas, midas_transform, img):
  # image_rgb is HxWx3 uint8
  input_batch = midas_transform(img).to(device)

  with torch.no_grad():
      prediction = midas(input_batch)

  depth = prediction.squeeze().cpu().numpy()

  # Resize to original image size
  depth = cv2.resize(depth, (img1.shape[1], img1.shape[0]))
  return depth

## Step 5: Calibrate to height above plate

## Step 6: Calculate volume

In [ ]:
def s6_calculate_volume(depth_mm, plate_mask, plate_diameter_mm):
    """
    Compute volume from a depth/height map over a circular plate region.

    Assumptions:
      - depth_mm is per-pixel "height above table" (mm) OR "depth to fill" (mm) —
        i.e., it should be nonnegative for volume contribution.
      - plate_mask marks the plate region (boolean or 0..1 float soft mask).
      - plate_diameter_mm is the real plate diameter corresponding to the plate region.

    Returns:
      volume_ml: float  # 1 mL == 1 cm^3 == 1000 mm^3
      volume_mm3: float
      pixel_area_mm2: float
    """
    depth = np.asarray(depth_mm, dtype=np.float32)
    mask = np.asarray(plate_mask, dtype=np.float32)

    if depth.shape != mask.shape:
        raise ValueError(f"depth_mm and plate_mask must have the same shape, got {depth.shape} vs {mask.shape}")

    if plate_diameter_mm <= 0:
        raise ValueError("plate_diameter_mm must be > 0")

    # Clamp inputs (volume can't be negative; mask is a weight in [0,1])
    depth = np.maximum(depth, 0.0)
    mask = np.clip(mask, 0.0, 1.0)

    # Estimate per-pixel physical area by matching mask's area to the plate's real area.
    # This avoids needing mm_per_px explicitly.
    plate_area_mm2 = np.pi * (plate_diameter_mm * 0.5) ** 2
    mask_area_px = float(mask.sum())

    if mask_area_px <= 1e-6:
        return 0.0, 0.0, 0.0

    pixel_area_mm2 = plate_area_mm2 / mask_area_px

    # Volume = sum(depth_mm * area_mm2) over pixels => mm^3
    volume_mm3 = float((depth * mask).sum()) * float(pixel_area_mm2)

    # Convert mm^3 -> mL (1 mL = 1000 mm^3)
    volume_ml = volume_mm3 / 1000.0

    return volume_ml, volume_mm3, pixel_area_mm2

## Steps: Execution

In [ ]:
img1 = s1_upload_preprocess_image()

print(f"img1 shape: {img1.shape}")
show_image(img1, "Image 1")


In [ ]:
plate_mask1 = s2_detect_plate_mask(img1)

show_image(overlay_mask(img1, plate_mask1), "Overlay 1")

# fig = plot_heatmap_overlay(plate_mask1, img1=img1)
# fig.show()

In [ ]:
plate_mask_u8 = _to_u8_mask(plate_mask1)

H, W = plate_mask_u8.shape

# ---- Fit ellipse from plate mask to get center + diameter in pixels ----
cx, cy, plate_diameter_px, ellipse = ellipse_params_from_mask(plate_mask_u8)

# ---- MEASURED-FOR-A-FACT plate parameters (EDIT THESE) ----
OUTER_DIAMETER_MM = 272.5        # e.g. 260.0
INNER_RADIUS_MM = 177/2          # flat inner region radius
LIP_END_RADIUS_MM = 188/2         # radius where lip ends
SLOPE_END_RADIUS_MM = 272.5/2      # radius where slope ends
# (outer radius implied = OUTER_DIAMETER_MM/2)

# Heights above table
H_INNER_MM = 9.0                 # if you define table as 0 and plate inner surface is above it, set that here
H_LIP_MM = 13.0                   # height at lip end
H_SLOPE_END_MM = 20.0             # height at slope end / outer band

LIP_PROFILE = "smooth"
SLOPE_PROFILE = "linear"

# Feathering for mask blending (px)
FEATHER_PX = 2.0


empty_plate_mm_map, empty_plate_rel_map = s3_empty_plate_height_map(
    height_px=H,
    width_px=W,
    center_xy_px=(cx, cy),
    plate_diameter_px=plate_diameter_px,
    outer_diameter_mm=OUTER_DIAMETER_MM,
    inner_radius_mm=INNER_RADIUS_MM,
    lip_end_radius_mm=LIP_END_RADIUS_MM,
    slope_end_radius_mm=SLOPE_END_RADIUS_MM,
    h_inner_mm=H_INNER_MM,
    h_lip_mm=H_LIP_MM,
    h_slope_end_mm=H_SLOPE_END_MM,
    lip_profile=LIP_PROFILE,
    slope_profile=SLOPE_PROFILE,
    feather_px=FEATHER_PX,
)

fig = plot_heatmap_overlay(empty_plate_rel_map, img1=img1)
fig.show()

In [ ]:
midas_depth = s4_run_midas(MIDAS, MIDAS_TRANSFORM, img1)

In [ ]:
midas_adj = midas_depth - 8 - (empty_plate_rel_map*5.5)
midas_adj = midas_adj * (midas_adj >= 7)
midas_adj_mask = midas_adj > 0

In [ ]:
fig = plot_heatmap_overlay(midas_adj, img1=img1, mask=midas_adj_mask)
fig.show()

In [ ]:
volume = s6_calculate_volume(midas_adj, plate_mask1, 272.5)

print(volume)